In [3]:
import pandas as pd
df = pd.read_csv('../data/traveler_data.csv', encoding='cp949')

## 방한횟수
D_NUM → 재방문여부

기존변수: 1회 / 2회 / 3회 / 4회 이상

새로운변수: 0 = 1회, 1 = 2회 이상(재방문)

In [4]:
df_new = pd.DataFrame()

In [3]:
df_new['재방문여부'] = df['D_NUM'].apply(lambda x: 1 if x >= 2 else 0)

In [4]:
df_new

,재방문여부
0,1
1,1
2,1
3,0
4,1
...,...
16211,1
16212,1
16213,0
16214,1


## 여행형태
TYP → 여행형태_개별여행, 여행형태_에어텔, 여행형태_단체여행

기존변수: 1 = 개별여행 / 2 = Air-tel 패키지 / 3 = 단체여행

새로운변수: 각 항목을 원핫인코딩 → 열로 분리

In [5]:
# TYP 원핫인코딩 후 한글 열 이름 지정
df_typ = pd.get_dummies(df['TYP'])
df_typ.columns = ['여행형태_개별여행', '여행형태_에어텔', '여행형태_단체여행']

# df_new에 병합
df_new = pd.concat([df_new, df_typ], axis=1)

In [6]:
df_new

,재방문여부,여행형태_개별여행,여행형태_에어텔,여행형태_단체여행
0,1,True,False,False
1,1,True,False,False
2,1,True,False,False
3,0,True,False,False
4,1,True,False,False
...,...,...,...,...
16211,1,True,False,False
16212,1,True,False,False
16213,0,True,False,False
16214,1,True,False,False


## 주요 목적

Q1 → Q1_응답대상

In [7]:
df_new['Q1_응답대상'] = df['Q1'].apply(lambda x: 1 if x in [1, 2] else 0)

In [8]:
df_new.isna().sum()

재방문여부        0
여행형태_개별여행    0
여행형태_에어텔     0
여행형태_단체여행    0
Q1_응답대상      0
dtype: int64

## 한국여행 관심계기

Q1_1a1 → Q1_관심계기_그룹, Q1_관심계기_{그룹명}

기존변수: 1~11 (관심 계기 항목, 1순위)

새로운변수: '새로움', '경험', '행사', '경제성', '접근성', '문화콘텐츠', '정보노출', '기타'

| 그룹명     | 포함 항목 코드 | 설명            |
| ------- | -------- | ------------- |
| `새로움`  | 1        | 단순한 새로운 장소 욕구 |
| `경험`  | 2        | 과거 만족에 기반     |
| `행사`  | 3        | 가족행사 등 이벤트 목적 |
| `경제성`   | 4, 10    | 가격, 경비 측면     |
| `접근성`   | 5, 6     | 거리, 항공편 등     |
| `문화콘텐츠` | 7, 8     | 한류, 전통문화 포함   |
| `정보노출`  | 9        | 관광정보 인식       |
| `기타`    | 11       | 기타 응답         |


In [9]:
df_new['Q1_1a1'] = df['Q1_1a1']

# 1단계: 그룹 문자열로 변환
def map_q1_group(row):
    if row['Q1_응답대상'] != 1:
        return '응답대상외'
    
    x = row['Q1_1a1']
    if x == 1:
        return '새로움'
    elif x == 2:
        return '경험'
    elif x == 3:
        return '행사'
    elif x in [4, 10]:
        return '경제성'
    elif x in [5, 6]:
        return '접근성'
    elif x in [7, 8]:
        return '문화콘텐츠'
    elif x == 9:
        return '정보노출'
    elif x == 11:
        return '기타'

df_new['Q1_관심계기_그룹'] = df_new.apply(map_q1_group, axis=1)

# 2단계: 원핫인코딩
df_group_dummy = pd.get_dummies(df_new['Q1_관심계기_그룹'], prefix='Q1_관심계기')

# 3단계: df_new에 붙이기
df_new = pd.concat([df_new, df_group_dummy], axis=1)


In [10]:
df_new

,재방문여부,여행형태_개별여행,여행형태_에어텔,여행형태_단체여행,Q1_응답대상,Q1_1a1,Q1_관심계기_그룹,Q1_관심계기_경제성,Q1_관심계기_경험,Q1_관심계기_기타,Q1_관심계기_문화콘텐츠,Q1_관심계기_새로움,Q1_관심계기_응답대상외,Q1_관심계기_접근성,Q1_관심계기_정보노출,Q1_관심계기_행사
0,1,True,False,False,1,7.0,문화콘텐츠,False,False,False,True,False,False,False,False,False
1,1,True,False,False,1,7.0,문화콘텐츠,False,False,False,True,False,False,False,False,False
2,1,True,False,False,1,7.0,문화콘텐츠,False,False,False,True,False,False,False,False,False
3,0,True,False,False,1,8.0,문화콘텐츠,False,False,False,True,False,False,False,False,False
4,1,True,False,False,1,2.0,경험,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16211,1,True,False,False,1,5.0,접근성,False,False,False,False,False,False,True,False,False
16212,1,True,False,False,1,9.0,정보노출,False,False,False,False,False,False,False,True,False
16213,0,True,False,False,1,5.0,접근성,False,False,False,False,False,False,True,False,False
16214,1,True,False,False,1,2.0,경험,False,True,False,False,False,False,False,False,False


In [11]:
df_new.isna().sum()

재방문여부               0
여행형태_개별여행           0
여행형태_에어텔            0
여행형태_단체여행           0
Q1_응답대상             0
Q1_1a1           3955
Q1_관심계기_그룹          0
Q1_관심계기_경제성         0
Q1_관심계기_경험          0
Q1_관심계기_기타          0
Q1_관심계기_문화콘텐츠       0
Q1_관심계기_새로움         0
Q1_관심계기_응답대상외       0
Q1_관심계기_접근성         0
Q1_관심계기_정보노출        0
Q1_관심계기_행사          0
dtype: int64

## 고려한 관광활동  
`Q3_1a1` → `Q1_고려한 관광활동_그룹` → `Q1_고려한 관광활동_{그룹명}`

기존변수:  
`Q3_1a1`: 1~17 (관광활동 1순위), 단 `Q1_응답대상 == 1`인 경우만 유효  

새로운변수:  
- `Q3_관광활동_그룹`: 그룹명 문자열 (예: 미식쇼핑, 자연휴식 등)  
- `Q3_관광활동_{그룹명}`: 이진변수

| 그룹명     | 코드 포함 항목 | 설명               |
| ------- | -------- | ---------------- |
| `미식`  | 1    | 음식과 쇼핑 위주 활동     |
| `쇼핑`  | 2     | 음식과 쇼핑 위주 활동     |
| `자연휴식`  | 3, 4     | 경관 감상 및 휴양 목적    |
| `전통문화`  | 5, 6, 7  | 고궁, 전통문화, 박물관    |
| `한류` | 8        | K-POP, 드라마 등     |
| `공연/축제` | 9, 10    | 공연, 축제 등 참여형 콘텐츠 |
| `오락/유흥` | 11, 12   | 유흥 및 테마파크 중심     |
| `미용/치료` | 13, 14   | 뷰티, 건강 관련 목적     |
| `스포츠`   | 15, 16   | 스포츠 관람/참가        |
| `기타`    | 17       | 기타 응답            |


In [12]:
# 먼저 Q3_1a1를 df_new로 복사
df_new['Q3_1a1'] = df['Q3_1a1']

# 그룹 매핑 함수 정의
def map_q3_group(row):
    if row['Q1_응답대상'] != 1:
        return '응답대상외'

    x = row['Q3_1a1']
    if x == 1:
        return '미식'
    elif x == 2:
        return '쇼핑'
    elif x in [3, 4]:
        return '자연휴식'
    elif x in [5, 6, 7]:
        return '전통문화'
    elif x == 8:
        return '한류'
    elif x in [9, 10]:
        return '공연/축제'
    elif x in [11, 12]:
        return '오락/유흥'
    elif x in [13, 14]:
        return '미용/치료'
    elif x in [15, 16]:
        return '스포츠'
    elif x == 17:
        return '기타'
    else:
        return '결측'

# 그룹 변수 생성
df_new['Q1_고려한 관광활동_그룹'] = df_new.apply(map_q3_group, axis=1)

# 원핫인코딩
df_q3_dummy = pd.get_dummies(df_new['Q1_고려한 관광활동_그룹'], prefix='Q1_고려한 관광활동')

# 병합
df_new = pd.concat([df_new, df_q3_dummy], axis=1)

In [13]:
df_new.isna().sum()

재방문여부                   0
여행형태_개별여행               0
여행형태_에어텔                0
여행형태_단체여행               0
Q1_응답대상                 0
Q1_1a1               3955
Q1_관심계기_그룹              0
Q1_관심계기_경제성             0
Q1_관심계기_경험              0
Q1_관심계기_기타              0
Q1_관심계기_문화콘텐츠           0
Q1_관심계기_새로움             0
Q1_관심계기_응답대상외           0
Q1_관심계기_접근성             0
Q1_관심계기_정보노출            0
Q1_관심계기_행사              0
Q3_1a1               3955
Q1_고려한 관광활동_그룹          0
Q1_고려한 관광활동_공연/축제       0
Q1_고려한 관광활동_기타          0
Q1_고려한 관광활동_미식          0
Q1_고려한 관광활동_미용/치료       0
Q1_고려한 관광활동_쇼핑          0
Q1_고려한 관광활동_스포츠         0
Q1_고려한 관광활동_오락/유흥       0
Q1_고려한 관광활동_응답대상외       0
Q1_고려한 관광활동_자연휴식        0
Q1_고려한 관광활동_전통문화        0
Q1_고려한 관광활동_한류          0
dtype: int64

## 고려한 관광 인프라

Q3_2a1 → Q3_관광인프라_그룹 → Q3_관광인프라_{그룹명}

기존변수:

Q3_2a1: 1~12 (관광 인프라 관련 항목, 1순위), 단 Q1_응답대상 == 1인 경우만 유효

새로운변수:

Q3_관광인프라_그룹: 그룹명 문자열 

Q3_관광인프라_{그룹명}: 이진 변수 (원핫인코딩)

| 그룹명     | 코드 포함 항목      | 설명                       |
| ------- | ------------- | ------------------------ |
| `기초편의`  | 1, 5, 6, 8 | 입출국·숙소·교통·길찾기 등 기본 편의 |
| `언어`  | 2 | 의사소통 |
| `안전/건강` | 3, 7          | 치안과 의료, 질병관리 관련 항목       |
| `정보접근`  | 9, 10, 11     | 관광안내, 인터넷, 디지털 정보 등      |
| `경제성`   | 4             | 경비 관련                    |
| `기타`    | 12            | 기타 응답                    |


In [14]:
# 먼저 Q3_2a1 원본 변수 복사
df_new['Q3_2a1'] = df['Q3_2a1']

# 그룹 매핑 함수 정의
def map_q3infra_group(row):
    if row['Q1_응답대상'] != 1:
        return '응답대상외'

    x = row['Q3_2a1']
    if x in [1, 5, 6, 8]:
        return '기초편의'
    elif x == 2:
        return '언어'
    elif x in [3, 7]:
        return '안전/건강'
    elif x in [9, 10, 11]:
        return '정보접근'
    elif x == 4:
        return '경제성'
    elif x == 12:
        return '기타'
    else:
        return '결측'

# 그룹 변수 생성
df_new['Q1_관광인프라_그룹'] = df_new.apply(map_q3infra_group, axis=1)

# 원핫인코딩
df_q3infra_dummy = pd.get_dummies(df_new['Q1_관광인프라_그룹'], prefix='Q1_관광인프라')

# 병합
df_new = pd.concat([df_new, df_q3infra_dummy], axis=1)


In [15]:
df_new.isna().sum()

재방문여부                   0
여행형태_개별여행               0
여행형태_에어텔                0
여행형태_단체여행               0
Q1_응답대상                 0
Q1_1a1               3955
Q1_관심계기_그룹              0
Q1_관심계기_경제성             0
Q1_관심계기_경험              0
Q1_관심계기_기타              0
Q1_관심계기_문화콘텐츠           0
Q1_관심계기_새로움             0
Q1_관심계기_응답대상외           0
Q1_관심계기_접근성             0
Q1_관심계기_정보노출            0
Q1_관심계기_행사              0
Q3_1a1               3955
Q1_고려한 관광활동_그룹          0
Q1_고려한 관광활동_공연/축제       0
Q1_고려한 관광활동_기타          0
Q1_고려한 관광활동_미식          0
Q1_고려한 관광활동_미용/치료       0
Q1_고려한 관광활동_쇼핑          0
Q1_고려한 관광활동_스포츠         0
Q1_고려한 관광활동_오락/유흥       0
Q1_고려한 관광활동_응답대상외       0
Q1_고려한 관광활동_자연휴식        0
Q1_고려한 관광활동_전통문화        0
Q1_고려한 관광활동_한류          0
Q3_2a1               3955
Q1_관광인프라_그룹             0
Q1_관광인프라_경제성            0
Q1_관광인프라_기초편의           0
Q1_관광인프라_기타             0
Q1_관광인프라_안전/건강          0
Q1_관광인프라_언어             0
Q1_관광인프라_응답대상외          0
Q1_관광인프라_정보접근           0
dtype: int64

In [16]:
df['Q4a1'].isna().sum()

np.int64(804)

##  여행 전 한국 관련 정보 수집 경로

| 그룹명        | 코드 포함 항목 | 설명                  |
| ---------- | -------- | ------------------- |
| `온라인`      | 1        | 인터넷/앱               |
| `오프라인`     | 2, 3, 5  | 여행사, 언론, 서적 등 전통 매체 |
| `지인` | 4        | 친지, 친구, 동료           |
| `상업기관`     | 6        | 항공사, 호텔 등           |
| `공공기관`     | 7        | 관광공사, 대사관 등         |
| `기타`       | 8        | 기타                  |
| `정보수집안함`      | 결측  | 정보를 얻지 않음           |


In [17]:
# 원본 변수 복사
df_new['Q4a1'] = df['Q4a1']

# 그룹 매핑 함수 정의
def map_q4_group(x):
    # if pd.isna(x):
    #     return '정보수집안함'
    if x == 1:
        return '온라인'
    elif x in [2, 3, 5]:
        return '오프라인'
    elif x == 4:
        return '지인'
    elif x == 6:
        return '상업기관'
    elif x == 7:
        return '공공기관'
    elif x == 8:
        return '기타'
    else:
        return '수집안함'

# 그룹 변수 생성
df_new['정보수집_그룹'] = df_new['Q4a1'].apply(map_q4_group)

# 원핫인코딩
df_q4_dummy = pd.get_dummies(df_new['정보수집_그룹'], prefix='정보수집')

# 병합
df_new = pd.concat([df_new, df_q4_dummy], axis=1)

In [18]:
df_new.isna().sum()

재방문여부                   0
여행형태_개별여행               0
여행형태_에어텔                0
여행형태_단체여행               0
Q1_응답대상                 0
Q1_1a1               3955
Q1_관심계기_그룹              0
Q1_관심계기_경제성             0
Q1_관심계기_경험              0
Q1_관심계기_기타              0
Q1_관심계기_문화콘텐츠           0
Q1_관심계기_새로움             0
Q1_관심계기_응답대상외           0
Q1_관심계기_접근성             0
Q1_관심계기_정보노출            0
Q1_관심계기_행사              0
Q3_1a1               3955
Q1_고려한 관광활동_그룹          0
Q1_고려한 관광활동_공연/축제       0
Q1_고려한 관광활동_기타          0
Q1_고려한 관광활동_미식          0
Q1_고려한 관광활동_미용/치료       0
Q1_고려한 관광활동_쇼핑          0
Q1_고려한 관광활동_스포츠         0
Q1_고려한 관광활동_오락/유흥       0
Q1_고려한 관광활동_응답대상외       0
Q1_고려한 관광활동_자연휴식        0
Q1_고려한 관광활동_전통문화        0
Q1_고려한 관광활동_한류          0
Q3_2a1               3955
Q1_관광인프라_그룹             0
Q1_관광인프라_경제성            0
Q1_관광인프라_기초편의           0
Q1_관광인프라_기타             0
Q1_관광인프라_안전/건강          0
Q1_관광인프라_언어             0
Q1_관광인프라_응답대상외          0
Q1_관광인프라_정보접근           0
Q4a1        

## 부족했던 정보

Q4_2a1 → Q4_부족정보_그룹 → Q4_부족정보_{그룹명}

기존변수:
1~11 (부족했던 정보 1순위), 결측은 “충분함”으로 간주

새로운변수:

Q4_부족정보_그룹: 그룹명 문자열 (예: 음식/쇼핑, 생활정보 등)

Q4_부족정보_{그룹명}: 이진 변수 (원핫인코딩)

| 그룹명     | 코드 포함 항목 | 설명             |
| ------- | -------- | -------------- |
| `관광지정보` | 1        | 장소, 운영정보 등     |
| `교통정보`  | 2        | 거리, 이동 동선 등    |
| `음식/쇼핑` | 3, 4     | 먹거리와 쇼핑 관련 정보  |
| `숙박`    | 5        | 숙소 정보          |
| `문화/행사` | 6, 7     | 축제, 한류 문화 체험 등 |
| `생활정보`  | 8, 9     | 금융, 통신 등 생활편의  |
| `기타`    | 10       | 기타 응답          |
| `충분함`   | 결측 | 부족한 정보 없음      |


In [19]:
# 원본 변수 복사
df_new['Q4_2a1'] = df['Q4_2a1']

# 그룹 매핑 함수 정의
def map_q4lack_group(x):
    if pd.isna(x) or x == 11:
        return '충분함'
    elif x == 1:
        return '관광지정보'
    elif x == 2:
        return '교통정보'
    elif x in [3, 4]:
        return '음식/쇼핑'
    elif x == 5:
        return '숙박'
    elif x in [6, 7]:
        return '문화/행사'
    elif x in [8, 9]:
        return '생활정보'
    elif x == 10:
        return '기타'
    else:
        return '결측'

# 그룹 변수 생성
df_new['부족했던 정보_그룹'] = df_new['Q4_2a1'].apply(map_q4lack_group)

# 원핫인코딩
df_q4lack_dummy = pd.get_dummies(df_new['부족했던 정보_그룹'], prefix='부족했던 정보')

# 병합
df_new = pd.concat([df_new, df_q4lack_dummy], axis=1)

In [20]:
df_new.isna().sum()

재방문여부                   0
여행형태_개별여행               0
여행형태_에어텔                0
여행형태_단체여행               0
Q1_응답대상                 0
Q1_1a1               3955
Q1_관심계기_그룹              0
Q1_관심계기_경제성             0
Q1_관심계기_경험              0
Q1_관심계기_기타              0
Q1_관심계기_문화콘텐츠           0
Q1_관심계기_새로움             0
Q1_관심계기_응답대상외           0
Q1_관심계기_접근성             0
Q1_관심계기_정보노출            0
Q1_관심계기_행사              0
Q3_1a1               3955
Q1_고려한 관광활동_그룹          0
Q1_고려한 관광활동_공연/축제       0
Q1_고려한 관광활동_기타          0
Q1_고려한 관광활동_미식          0
Q1_고려한 관광활동_미용/치료       0
Q1_고려한 관광활동_쇼핑          0
Q1_고려한 관광활동_스포츠         0
Q1_고려한 관광활동_오락/유흥       0
Q1_고려한 관광활동_응답대상외       0
Q1_고려한 관광활동_자연휴식        0
Q1_고려한 관광활동_전통문화        0
Q1_고려한 관광활동_한류          0
Q3_2a1               3955
Q1_관광인프라_그룹             0
Q1_관광인프라_경제성            0
Q1_관광인프라_기초편의           0
Q1_관광인프라_기타             0
Q1_관광인프라_안전/건강          0
Q1_관광인프라_언어             0
Q1_관광인프라_응답대상외          0
Q1_관광인프라_정보접근           0
Q4a1        

## 동반자 유무

Q7A → 동반자유무

기존변수:

1 = 없음, 2 = 있음

새로운변수:

1 = 있음, 0 = 없음

In [21]:
# 동반자유무: 0 = 없음, 1 = 있음
df_new['동반자유무'] = df['Q7A'].apply(lambda x: 1 if x == 2 else 0)

In [22]:
df_new.isna().sum()

재방문여부                   0
여행형태_개별여행               0
여행형태_에어텔                0
여행형태_단체여행               0
Q1_응답대상                 0
Q1_1a1               3955
Q1_관심계기_그룹              0
Q1_관심계기_경제성             0
Q1_관심계기_경험              0
Q1_관심계기_기타              0
Q1_관심계기_문화콘텐츠           0
Q1_관심계기_새로움             0
Q1_관심계기_응답대상외           0
Q1_관심계기_접근성             0
Q1_관심계기_정보노출            0
Q1_관심계기_행사              0
Q3_1a1               3955
Q1_고려한 관광활동_그룹          0
Q1_고려한 관광활동_공연/축제       0
Q1_고려한 관광활동_기타          0
Q1_고려한 관광활동_미식          0
Q1_고려한 관광활동_미용/치료       0
Q1_고려한 관광활동_쇼핑          0
Q1_고려한 관광활동_스포츠         0
Q1_고려한 관광활동_오락/유흥       0
Q1_고려한 관광활동_응답대상외       0
Q1_고려한 관광활동_자연휴식        0
Q1_고려한 관광활동_전통문화        0
Q1_고려한 관광활동_한류          0
Q3_2a1               3955
Q1_관광인프라_그룹             0
Q1_관광인프라_경제성            0
Q1_관광인프라_기초편의           0
Q1_관광인프라_기타             0
Q1_관광인프라_안전/건강          0
Q1_관광인프라_언어             0
Q1_관광인프라_응답대상외          0
Q1_관광인프라_정보접근           0
Q4a1        

## 동반자 유형

Q7a2 ~ Q7a8 → 동반자유형_그룹 → 동반자유형_{그룹명}

기존변수:
Q7a2 ~ Q7a8: 동반자 유형 코드 (1~8), 다중응답

새로운변수:

동반자유형_그룹: 포함된 코드에 따라 그룹명 문자열

동반자유형_1인, 동반자유형_배우자, ..., 동반자유형_기타: 원핫 인코딩

| 그룹명      | 코드 포함 항목 | 설명            |
| -------- | -------- | ------------- |
| `1인`   | 1        | 혼자 방문         |
| `배우자` | 2     | 배우자   |
| `자녀` | 4     | 자녀    |
| `부모/친지`  | 3, 5     | 부모님, 기타 가족/친지 |
| `친구/동료`  | 6, 7     | 친구 또는 직장 동료   |
| `기타`     | 8        | 기타 동반자        |


In [23]:
# 다중응답 열 리스트
q7_cols = ['Q7a2', 'Q7a3', 'Q7a4', 'Q7a5', 'Q7a6', 'Q7a7', 'Q7a8']

# 그룹핑 함수 (Q7a_dk 포함)
def extract_q7_group(row):
    group_set = set()

    # Q7a_dk = 1인 경우 → 혼자 방문
    if row.get('Q7a_dk') == 1:
        group_set.add('1인')

    # 다중응답 코드들 처리
    codes = [row[col] for col in q7_cols if not pd.isna(row[col])]
    for code in codes:
        if code == 1:
            group_set.add('혼자')
        elif code == 2:
            group_set.add('배우자')
        elif code == 4:
            group_set.add('자녀')
        elif code in [3, 5]:
            group_set.add('부모/친지')
        elif code in [6, 7]:
            group_set.add('친구/동료')
        elif code == 8:
            group_set.add('기타')

    return list(group_set)

# 적용
df_new['동반자유형_그룹리스트'] = df.apply(extract_q7_group, axis=1)

# 원핫인코딩
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
q7_group_dummy = pd.DataFrame(mlb.fit_transform(df_new['동반자유형_그룹리스트']),
                              columns=[f'동반자유형_{cls}' for cls in mlb.classes_],
                              index=df_new.index)

# 병합
df_new = pd.concat([df_new, q7_group_dummy], axis=1)


In [24]:
df_new.isna().sum()

재방문여부          0
여행형태_개별여행      0
여행형태_에어텔       0
여행형태_단체여행      0
Q1_응답대상        0
              ..
동반자유형_기타       0
동반자유형_배우자      0
동반자유형_부모/친지    0
동반자유형_자녀       0
동반자유형_친구/동료    0
Length: 65, dtype: int64

In [25]:
df_new['동반자유형_그룹리스트']

0              [배우자]
1          [자녀, 배우자]
2        [자녀, 친구/동료]
3               [1인]
4            [부모/친지]
            ...     
16211        [친구/동료]
16212          [배우자]
16213           [자녀]
16214          [배우자]
16215          [배우자]
Name: 동반자유형_그룹리스트, Length: 16216, dtype: object

## 만족한 활동

Q8_1a1 → 만족활동_그룹 → 만족활동_{그룹명}

기존변수:
Q8_1a1: 1~20 (만족한 활동 1순위)

새로운변수:

만족활동_그룹: 그룹명 문자열

만족활동_미식, 만족활동_쇼핑, ..., 만족활동_기타: 이진 변수 (원핫인코딩)

| 그룹명       | 코드 포함 항목   | 설명              |
| --------- | ---------- | --------------- |
| `미식`      | 1          | 음식 관련 관광        |
| `쇼핑`      | 2          | 쇼핑              |
| `자연/휴식`   | 3, 4       | 자연경관 및 휴양/웰니스   |
| `전통문화`    | 5, 6, 7    | 역사, 전통체험, 박물관 등 |
| `한류`      | 8          | K-POP, 드라마      |
| `공연/축제`   | 9, 10      | 공연 관람, 지역 축제    |
| `오락/유흥`   | 11, 12     | 나이트라이프, 테마파크    |
| `미용/치료`   | 13, 14     | 뷰티/의료 목적 방문     |
| `스포츠`     | 15, 16     | 스포츠 관람 및 참가     |
| `비즈니스/학술` | 17, 18, 19 | 국제회의, 산업 시찰, 연수 |
| `기타`      | 20         | 기타 응답           |

In [26]:
# 원본 변수 복사
df_new['Q8_1a1'] = df['Q8_1a1']

# 그룹 매핑 함수
def map_q8_group(x):
    if pd.isna(x):
        return '결측'
    elif x == 1:
        return '미식'
    elif x == 2:
        return '쇼핑'
    elif x in [3, 4]:
        return '자연/휴식'
    elif x in [5, 6, 7]:
        return '전통문화'
    elif x == 8:
        return '한류'
    elif x in [9, 10]:
        return '공연/축제'
    elif x in [11, 12]:
        return '오락/유흥'
    elif x in [13, 14]:
        return '미용/치료'
    elif x in [15, 16]:
        return '스포츠'
    elif x in [17, 18, 19]:
        return '비즈니스/학술'
    elif x == 20:
        return '기타'
    else:
        return '결측'

# 그룹 변수 생성
df_new['만족활동_그룹'] = df_new['Q8_1a1'].apply(map_q8_group)

# 원핫인코딩
df_q8_dummy = pd.get_dummies(df_new['만족활동_그룹'], prefix='만족활동')

# 병합
df_new = pd.concat([df_new, df_q8_dummy], axis=1)

In [27]:
df_new.isna().sum()

재방문여부         0
여행형태_개별여행     0
여행형태_에어텔      0
여행형태_단체여행     0
Q1_응답대상       0
             ..
만족활동_스포츠      0
만족활동_오락/유흥    0
만족활동_자연/휴식    0
만족활동_전통문화     0
만족활동_한류       0
Length: 78, dtype: int64

## 방문 지역 조합

KWONA1, KWONA2, KWONA3 → 방문지역조합

기존변수:

KWONA1 = 1 → 수도권만

KWONA2 = 2 → 지방만

KWONA3 = 3 → 수도권+지방

새로운변수:

방문지역조합: 문자열형 구분 (수도권만, 지방만, 수도권+지방)

원핫인코딩: 방문지역조합_수도권만, 방문지역조합_지방만, 방문지역조합_수도권+지방

In [28]:
# 지역 조합 판별 함수
def map_region_combo(row):
    if row.get('KWONA1') == 1:
        return '수도권만'
    elif row.get('KWONA2') == 2:
        return '지방만'
    elif row.get('KWONA3') == 3:
        return '수도권+지방'
    else:
        return '결측'

# 변수 생성
df_new['방문지역조합'] = df.apply(map_region_combo, axis=1)

# 원핫인코딩
df_region_dummy = pd.get_dummies(df_new['방문지역조합'], prefix='방문지역조합')

# 병합
df_new = pd.concat([df_new, df_region_dummy], axis=1)

In [29]:
df_new

,재방문여부,여행형태_개별여행,여행형태_에어텔,여행형태_단체여행,Q1_응답대상,Q1_1a1,Q1_관심계기_그룹,Q1_관심계기_경제성,Q1_관심계기_경험,Q1_관심계기_기타,...,만족활동_쇼핑,만족활동_스포츠,만족활동_오락/유흥,만족활동_자연/휴식,만족활동_전통문화,만족활동_한류,방문지역조합,방문지역조합_수도권+지방,방문지역조합_수도권만,방문지역조합_지방만
0,1,True,False,False,1,7.0,문화콘텐츠,False,False,False,...,True,False,False,False,False,False,수도권만,False,True,False
1,1,True,False,False,1,7.0,문화콘텐츠,False,False,False,...,False,False,False,False,False,False,수도권만,False,True,False
2,1,True,False,False,1,7.0,문화콘텐츠,False,False,False,...,False,False,True,False,False,False,수도권만,False,True,False
3,0,True,False,False,1,8.0,문화콘텐츠,False,False,False,...,False,False,True,False,False,False,수도권만,False,True,False
4,1,True,False,False,1,2.0,경험,False,True,False,...,True,False,False,False,False,False,수도권만,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16211,1,True,False,False,1,5.0,접근성,False,False,False,...,False,False,False,True,False,False,지방만,False,False,True
16212,1,True,False,False,1,9.0,정보노출,False,False,False,...,True,False,False,False,False,False,지방만,False,False,True
16213,0,True,False,False,1,5.0,접근성,False,False,False,...,False,False,False,False,False,False,지방만,False,False,True
16214,1,True,False,False,1,2.0,경험,False,True,False,...,False,False,False,True,False,False,지방만,False,False,True


## 총 체제기간

M일HAP → 총체재기간

기존변수:

M일HAP: 숫자형, 총 체재일 수

새로운변수:

총체재기간: 동일한 값, 변수명만 변경해서 사용

In [30]:
# 변수 복사
df_new['총체재기간'] = df['M일HAP']

In [31]:
(df['M일HAP'] == 0).sum()

np.int64(0)

체류일이 0인 경우는 없음

체류일별 소비금액 = (국제교통비 제외 총 소비금액/체류일) 이 식에 오류 없을것

In [32]:
df['M일HAP'].isna().sum()

np.int64(0)

## 1인 총소비_국제교통비 포함

In [5]:
df_new['1인 총소비_국제교통비 포함'] = df['총액1인TOT2']

In [6]:
df_new['1인 총소비_국제교통비 포함']

0        2710.00
1        1364.68
2        1065.90
3         922.68
4        1647.30
          ...   
16211     562.93
16212     335.52
16213    2677.35
16214     755.15
16215    2200.00
Name: 1인 총소비_국제교통비 포함, Length: 16216, dtype: float64

## 1인 총소비_국제교통비 제외

In [34]:
df_new['1인 총소비_국제교통비 제외'] = df['총액1인TOT_개별국제교통비제외2']

## 1인 체류일별 소비

In [35]:
df_new['1인 체류일별 소비금액'] = df['총액1인TOT_개별국제교통비제외2'] / df['M일HAP']

## 쇼핑장소



| 그룹명     | 코드 포함 항목 | 설명           |
| ------- | -------- | ------------ |
| `면세점`   | 1, 2     | 공항 및 시내 면세점  |
| `전통시장` | 3   | 전통시장   |
| `로드샵` | 4     | 로드샵 + 전통시장   |
| `백화점`  | 5| 백화점 |
| `대형유통`  | 6, 7  | 쇼핑몰, 마트 |
| `기타`    | 8        | 기타 응답        |


In [36]:
# 원본 변수 복사
df_new['Q10_3a1'] = df['Q10_3a1']

# 그룹 매핑 함수
def map_q10_shop_group(x):
    if pd.isna(x):
        return '쇼핑안함'
    elif x in [1, 2]:
        return '면세점'
    elif x == 3:
        return '전통시장'
    elif x == 4:
        return '로드샵'
    elif x == 5:
        return '백화점'  # 또는 '대형유통' 중 하나 선택 (동일 코드에 중복 그룹 불가)
    elif x in [6, 7]:
        return '대형유통'  # 또는 '대형유통' 중 하나 선택 (동일 코드에 중복 그룹 불가)
    elif x == 8:
        return '기타'
    # else:
    #     return '쇼핑안함'

# 그룹 변수 생성
df_new['쇼핑장소_그룹'] = df_new['Q10_3a1'].apply(map_q10_shop_group)

# 원핫인코딩
df_shop_dummy = pd.get_dummies(df_new['쇼핑장소_그룹'], prefix='쇼핑장소')

# 병합
df_new = pd.concat([df_new, df_shop_dummy], axis=1)


In [37]:
df_new['쇼핑장소_쇼핑안함'].sum()

np.int64(2293)

## 만족도

In [38]:
# 변수 복사
df_new['만족도'] = df['Q11']

## 재방문 의사

In [39]:
# 변수 복사
df_new['재방문의사'] = df['Q13']

## 월

In [40]:
# 변수 복사
df_new['월'] = df['D_MON']

## 분기

In [41]:
# 변수 복사
df_new['분기'] = df['D_BUN']

## 국가

In [42]:
# 코드 → 국가명 매핑 딕셔너리
country_map = {
    1: '중국',
    2: '일본',
    3: '대만',
    4: '미국',
    5: '홍콩',
    6: '태국',
    7: '베트남',
    8: '말레이시아',
    9: '필리핀',
    10: '싱가포르',
    11: '러시아',
    12: '중동',
    13: '인도네시아',
    14: '캐나다',
    15: '호주',
    16: '영국',
    17: '몽골',
    18: '독일',
    19: '프랑스',
    20: '인도',
    97: '기타'
}

# 국가명 파생 변수 생성
df_new['국가명'] = df['D_NAT'].map(country_map)


In [43]:
# 1단계: 국가명 더미화
df_country_dummy = pd.get_dummies(df_new['국가명'], prefix='국가')

# 2단계: 원본에 병합
df_new = pd.concat([df_new, df_country_dummy], axis=1)

## 연령

In [44]:
df_new['연령'] = df['D_BUN']

## 성별

D_SEX → 성별_남성, 성별_여성

기존변수:

D_SEX: 1 = 남성, 2 = 여성

새로운변수:

성별_남성, 성별_여성: 원핫인코딩된 이진 변수

In [45]:
# 성별 매핑 딕셔너리
sex_map = {
    1: '남성',
    2: '여성'
}

# 문자열 변수 생성
df_new['성별'] = df['D_SEX'].map(sex_map)

# 원핫인코딩
df_sex_dummy = pd.get_dummies(df_new['성별'], prefix='성별')
df_new = pd.concat([df_new, df_sex_dummy], axis=1)

## 가중치

In [ ]:
df_new['가중치'] = df['weight']

## 월/분기 더미화

In [47]:
import pandas as pd

# 월 더미화
month_dummies = pd.get_dummies(df_new["월"], prefix="월")

# 분기 더미화
quarter_dummies = pd.get_dummies(df_new["분기"], prefix="분기")

# 기존 데이터와 결합
df_new = pd.concat([df_new, month_dummies, quarter_dummies], axis=1)


## 파일 저장

In [48]:
df_new.columns.tolist()

['재방문여부',
 '여행형태_개별여행',
 '여행형태_에어텔',
 '여행형태_단체여행',
 'Q1_응답대상',
 'Q1_1a1',
 'Q1_관심계기_그룹',
 'Q1_관심계기_경제성',
 'Q1_관심계기_경험',
 'Q1_관심계기_기타',
 'Q1_관심계기_문화콘텐츠',
 'Q1_관심계기_새로움',
 'Q1_관심계기_응답대상외',
 'Q1_관심계기_접근성',
 'Q1_관심계기_정보노출',
 'Q1_관심계기_행사',
 'Q3_1a1',
 'Q1_고려한 관광활동_그룹',
 'Q1_고려한 관광활동_공연/축제',
 'Q1_고려한 관광활동_기타',
 'Q1_고려한 관광활동_미식',
 'Q1_고려한 관광활동_미용/치료',
 'Q1_고려한 관광활동_쇼핑',
 'Q1_고려한 관광활동_스포츠',
 'Q1_고려한 관광활동_오락/유흥',
 'Q1_고려한 관광활동_응답대상외',
 'Q1_고려한 관광활동_자연휴식',
 'Q1_고려한 관광활동_전통문화',
 'Q1_고려한 관광활동_한류',
 'Q3_2a1',
 'Q1_관광인프라_그룹',
 'Q1_관광인프라_경제성',
 'Q1_관광인프라_기초편의',
 'Q1_관광인프라_기타',
 'Q1_관광인프라_안전/건강',
 'Q1_관광인프라_언어',
 'Q1_관광인프라_응답대상외',
 'Q1_관광인프라_정보접근',
 'Q4a1',
 '정보수집_그룹',
 '정보수집_공공기관',
 '정보수집_기타',
 '정보수집_상업기관',
 '정보수집_수집안함',
 '정보수집_오프라인',
 '정보수집_온라인',
 '정보수집_지인',
 'Q4_2a1',
 '부족했던 정보_그룹',
 '부족했던 정보_관광지정보',
 '부족했던 정보_교통정보',
 '부족했던 정보_기타',
 '부족했던 정보_문화/행사',
 '부족했던 정보_생활정보',
 '부족했던 정보_숙박',
 '부족했던 정보_음식/쇼핑',
 '부족했던 정보_충분함',
 '동반자유무',
 '동반자유형_그룹리스트',
 '동반자유형_1인',
 '동반자유형_기타',
 '동반자유형_배우자',
 '동반자유형_부모/친지',
 '동

In [49]:
# 제외할 컬럼 리스트
cols_to_drop = ['Q1_1a1','Q3_1a1', 'Q3_2a1', 'Q4a1', 'Q4_2a1', 'Q8_1a1', 'Q10_3a1', '쇼핑장소_결측'
, 'Q3_1a1',]

# 실제 존재하는 컬럼만 필터링
cols_to_drop_valid = [col for col in cols_to_drop if col in df_new.columns]

df_export = df_new.drop(columns=cols_to_drop_valid)

In [50]:
df_export = df_new.loc[:, ~df_new.columns.duplicated()]

In [51]:
df_export = df_new.drop(columns=['Q1_1a1','Q3_1a1', 'Q3_2a1', 'Q4a1', 'Q4_2a1', 'Q8_1a1', 'Q10_3a1'])

In [52]:
df_export.columns.tolist()

['재방문여부',
 '여행형태_개별여행',
 '여행형태_에어텔',
 '여행형태_단체여행',
 'Q1_응답대상',
 'Q1_관심계기_그룹',
 'Q1_관심계기_경제성',
 'Q1_관심계기_경험',
 'Q1_관심계기_기타',
 'Q1_관심계기_문화콘텐츠',
 'Q1_관심계기_새로움',
 'Q1_관심계기_응답대상외',
 'Q1_관심계기_접근성',
 'Q1_관심계기_정보노출',
 'Q1_관심계기_행사',
 'Q1_고려한 관광활동_그룹',
 'Q1_고려한 관광활동_공연/축제',
 'Q1_고려한 관광활동_기타',
 'Q1_고려한 관광활동_미식',
 'Q1_고려한 관광활동_미용/치료',
 'Q1_고려한 관광활동_쇼핑',
 'Q1_고려한 관광활동_스포츠',
 'Q1_고려한 관광활동_오락/유흥',
 'Q1_고려한 관광활동_응답대상외',
 'Q1_고려한 관광활동_자연휴식',
 'Q1_고려한 관광활동_전통문화',
 'Q1_고려한 관광활동_한류',
 'Q1_관광인프라_그룹',
 'Q1_관광인프라_경제성',
 'Q1_관광인프라_기초편의',
 'Q1_관광인프라_기타',
 'Q1_관광인프라_안전/건강',
 'Q1_관광인프라_언어',
 'Q1_관광인프라_응답대상외',
 'Q1_관광인프라_정보접근',
 '정보수집_그룹',
 '정보수집_공공기관',
 '정보수집_기타',
 '정보수집_상업기관',
 '정보수집_수집안함',
 '정보수집_오프라인',
 '정보수집_온라인',
 '정보수집_지인',
 '부족했던 정보_그룹',
 '부족했던 정보_관광지정보',
 '부족했던 정보_교통정보',
 '부족했던 정보_기타',
 '부족했던 정보_문화/행사',
 '부족했던 정보_생활정보',
 '부족했던 정보_숙박',
 '부족했던 정보_음식/쇼핑',
 '부족했던 정보_충분함',
 '동반자유무',
 '동반자유형_그룹리스트',
 '동반자유형_1인',
 '동반자유형_기타',
 '동반자유형_배우자',
 '동반자유형_부모/친지',
 '동반자유형_자녀',
 '동반자유형_친구/동료',
 '만족활동_그룹',
 '만족활동_공연/축제',


In [53]:
df_export = df_export.drop(columns=['국가명','성별', '월', '분기', 'Q1_관광인프라_그룹', '정보수집_그룹', '부족했던 정보_그룹', '동반자유형_그룹리스트','만족활동_그룹', '방문지역조합', 'Q1_관심계기_그룹', 'Q1_고려한 관광활동_그룹'])

In [54]:
df_export.columns.tolist()

['재방문여부',
 '여행형태_개별여행',
 '여행형태_에어텔',
 '여행형태_단체여행',
 'Q1_응답대상',
 'Q1_관심계기_경제성',
 'Q1_관심계기_경험',
 'Q1_관심계기_기타',
 'Q1_관심계기_문화콘텐츠',
 'Q1_관심계기_새로움',
 'Q1_관심계기_응답대상외',
 'Q1_관심계기_접근성',
 'Q1_관심계기_정보노출',
 'Q1_관심계기_행사',
 'Q1_고려한 관광활동_공연/축제',
 'Q1_고려한 관광활동_기타',
 'Q1_고려한 관광활동_미식',
 'Q1_고려한 관광활동_미용/치료',
 'Q1_고려한 관광활동_쇼핑',
 'Q1_고려한 관광활동_스포츠',
 'Q1_고려한 관광활동_오락/유흥',
 'Q1_고려한 관광활동_응답대상외',
 'Q1_고려한 관광활동_자연휴식',
 'Q1_고려한 관광활동_전통문화',
 'Q1_고려한 관광활동_한류',
 'Q1_관광인프라_경제성',
 'Q1_관광인프라_기초편의',
 'Q1_관광인프라_기타',
 'Q1_관광인프라_안전/건강',
 'Q1_관광인프라_언어',
 'Q1_관광인프라_응답대상외',
 'Q1_관광인프라_정보접근',
 '정보수집_공공기관',
 '정보수집_기타',
 '정보수집_상업기관',
 '정보수집_수집안함',
 '정보수집_오프라인',
 '정보수집_온라인',
 '정보수집_지인',
 '부족했던 정보_관광지정보',
 '부족했던 정보_교통정보',
 '부족했던 정보_기타',
 '부족했던 정보_문화/행사',
 '부족했던 정보_생활정보',
 '부족했던 정보_숙박',
 '부족했던 정보_음식/쇼핑',
 '부족했던 정보_충분함',
 '동반자유무',
 '동반자유형_1인',
 '동반자유형_기타',
 '동반자유형_배우자',
 '동반자유형_부모/친지',
 '동반자유형_자녀',
 '동반자유형_친구/동료',
 '만족활동_공연/축제',
 '만족활동_기타',
 '만족활동_미식',
 '만족활동_미용/치료',
 '만족활동_비즈니스/학술',
 '만족활동_쇼핑',
 '만족활동_스포츠',
 '만족활동_오락/유흥',
 '만족활동_자연

In [55]:
# CSV 저장
df_export.to_csv('../data/최종모델링데이터_Final.csv', index=False, encoding='utf-8-sig')